# 本章简介
另一种常用于降维方法就是特征抽取，特征抽取可以将原始数据集变化到一个维度更低的新的特征子空间，从而对数据进行压缩，避免维度灾难  
本章读者将学习三种降维技术：  
+ 无监督数据压缩——主成分分析(PCA)
+ 基于类别可分最大化的监督降维技术——线性判别分析(LDA)
+ 通过核主成分分析进行非线性降维 5  

# 5.1 无监督数据降维技术-主成分分析
> 本章介绍了主成分分析(PCA)的原理、实现和使用方法

### 问题：特征选择和特征抽取有什么区别？
> 特征抽取后的新特征是原来特征的一个映射，而特征选择后的特征是原来特征的一个子集 5  

特征抽取算法会将数据转换或者映射到一个新的特征空间，可以理解为：在尽可能保持相关信息的情况下，对数据进行压缩的一种方法  
特征抽取通常用于提高计算效率，同样也可以帮助我们降低"维度灾难"——尤其当模型不适于正则化处理时  
主成分分析(PCA)是一种广泛应用于不同领域的无监督线性数据转换技术，其突出作用是降维  
简而言之，PCA的目标是在高维数据中找到最大方差的方向，并将数据映射到一个维度不大于原始数据的新的子空间上 5  
如下图所示，新的子空间上正交的坐标轴（主成分）可被解释为方差最大的方向。在此，$x_1$和$x_2$为原始特征的坐标轴，而PC1和PC2即为主成分  
![5-1](../syn_pic/py_machine_learning/5-1.png)
如果使用PCA降维，我们将构建一个$d\times k$维的转换矩阵W，这样就可以将一个样本向量x映射到一个新的k维特征子空间上去，此空间的维度小于原始的d维特征空间：  
$$x=[x_1,x_2,\dots,x_d]，x\in{R}^d$$
$$\downarrow xW,W\in{R}^{d\times k}$$
$$z=[z_1,z_2,\dots,z_k],z\in{R}^k$$
完成从原始d维数据到新的k维子空间(一般情况下$k\ll d$)的转换后，第一主成分的方差应该是最大的，由于各主成分之间是不相关的（正交的），后续各主成分也具备尽可能大的方差  
需要注意的是，主成分的方向对数据值的范围高度敏感，如果特征的值使用不同的度量标准，我们需要先对特征进行标准化处理  
我们先通过以下几个步骤来概括以下算法的流程：
1. 对原始d维数据集做标准化处理 5\*2    
2. 构造样本的协方差矩阵
3. 计算协方差矩阵的特征值和相应的特征向量
4. 选择与前k个最大特征值对应的特征向量，其中k为新特征空间的维度($k\le d$)
5. 通过前k个特征向量构建映射矩阵W
6. 通过映射矩阵W将d维的输入数据集X转换到新的k维特征子空间 5  

### 问题：如何理解主成分分析降维的原理？
> 主成分简单来说就是对结果影响最大的成分，而成分包含在各个特征中  
特征抽取的含义就是把成分从特征中抽取出来，将成分重新组合后映射到新的k维特征空间中  
这种抽取并组合的原理是通过计算贡献方差来实现的，贡献方差是基于特征向量和协方差矩阵的特征值计算出来 5  
  

## 5.1.1 总体方差与贡献方差
> 本节介绍了通过协方差的特征值和特征向量计算贡献方差的原理

本节，我们将学习主成分分析算法的前四个步骤：数据标准化、构造协方差矩阵、获得协方差矩阵的特征值和特征向量，以及按降序排列特征值所对应的特征向量  

In [ ]:
import pandas as pd
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                      header=None)
df_wine.columns = ['Class label', 'Alcohol', 
                   'Malic acid', 'Ash',
                  'Alcalinity of ash', 'Magnesium',
                  'Total phenols', 'Flavanoids',
                  'Nonflavanoid phenols',
                  'Proanthocyanins',
                  'Color intensity', 'Hue',
                  'OD280/OD315 of diluted wines',
                  'Proline']
df_wine.head()

我们将葡萄酒数据集划分为训练集和测试集，并使用单位方差将其标准化  

In [ ]:
'''
5
python	pandas	dataframe	values
python	sklearn	Model Selection	train_test_split()
python	sklearn	Preprocessing and Normalization	StandardScaler()
'''
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=0)
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

完成数据预处理后，我们进入第二步：构造协方差矩阵  
$d\times d$维协方差矩阵，其中d位数据集的维度，此矩阵成对地存储了不同特征之间地协方差，例如两个特征$x_j和x_k$ 可通过如下公式来计算协方差：   
$$\sigma_{jk}=\frac{1}{n}\sum_{i=1}^n{(x_j^{(i)}-\mu_j)(x_k^{(i)}-\mu_k)}$$
在此，$\mu_j$和$\mu_k$分别位特征j和k的均值。注意，我们做了标准化处理后，样本的均值将为零   
两个特征之间的协方差如果位正，说明它们会同时增减，而一个负的协方差值则表示两个特征会朝相反的方向变动，一个包含三个特征的协方差矩阵可记为：5  
$$\Sigma=\left[\begin{matrix}\sigma^2_1&\sigma_{12}&\sigma_{13} \cr 
    \sigma_{21}&\sigma^2_2&\sigma_{23} \cr 
    \sigma_{31}&\sigma_{32}&\sigma^2_3 \end{matrix}\right]$$
协方差矩阵的特征向量代表主成分（最大方差方向），而对应的特征值大小就决定了特征向量的重要性  
### 问题：协方差矩阵其特征向量、特征值分别是什么？
> 待定

就葡萄酒数据集来说，我们可以得到$13\times13$维协方差矩阵的13各特征向量及其对应的特征值。特征值V需满足如下条件：  
$$\Sigma{V}=\lambda{V}$$
5  
此处的特征值是$\lambda$一个标量，我们来计算协方差矩阵的特征对  
### 问题：什么是标量？
> 几何代数的概念，对应于矢量，标量亦称作无向量，用通俗的说法，标量是只有大小，没有方向的量 5  

In [ ]:
import numpy as np
cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
print('\nEigenvalues \n{:s}'.format(eigen_vals))